In [1]:
import pandas as pd

In [21]:
df = pd.read_csv('spotify_history.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149860 entries, 0 to 149859
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   spotify_track_uri  149860 non-null  object
 1   ts                 149860 non-null  object
 2   platform           149860 non-null  object
 3   ms_played          149860 non-null  int64 
 4   track_name         149860 non-null  object
 5   artist_name        149860 non-null  object
 6   album_name         149860 non-null  object
 7   reason_start       149717 non-null  object
 8   reason_end         149743 non-null  object
 9   shuffle            149860 non-null  bool  
 10  skipped            149860 non-null  bool  
dtypes: bool(2), int64(1), object(8)
memory usage: 10.6+ MB


In [22]:
df.drop_duplicates(keep='first', inplace=True)
df.shape[0]

148675

In [23]:
df.reason_start.value_counts()

reason_start
trackdone     75890
fwdbtn        53686
clickrow      10982
appload        3729
backbtn        2199
playbtn        1441
remote          433
trackerror      120
unknown          23
nextbtn          21
popup             5
endplay           2
autoplay          1
Name: count, dtype: int64

In [24]:
df.reason_end.value_counts()

reason_end
trackdone                       76380
fwdbtn                          53359
endplay                          9872
logout                           4364
backbtn                          2173
unexpected-exit-while-paused     1724
unknown                           268
remote                            224
unexpected-exit                   118
clickrow                           34
nextbtn                            21
appload                             7
popup                               6
reload                              4
trackerror                          4
Name: count, dtype: int64

In [25]:
df.loc[df.reason_start.isnull(), 'reason_start'] = 'unknown'
df.loc[df.reason_end.isnull(), 'reason_end'] = 'unknown'

- Если для song_info убирать дубликаты по всему сабсету, то останутся дубли по spotify_track_uri, так как песня может попадать в разные версии альбома.
- Например, у spotify_track_uri=3E7dfMvvCLUddWissuqMwr два альбома: "The Time Of Our Lives - International Version" и "	The Time Of Our Live"


In [46]:
song_info = df[['spotify_track_uri', 'track_name', 'artist_name', 'album_name']].drop_duplicates(subset='spotify_track_uri', 
                                                                                                 keep='first').reset_index(drop=True)
song_info.reset_index(inplace=True)
song_info.rename(columns={'index': 'song_id'}, inplace=True)

In [42]:
song_info.shape[0] == song_info.spotify_track_uri.nunique()

True

In [50]:
df = df.merge(song_info[['song_id', 'spotify_track_uri']], how='left', on='spotify_track_uri')
df.drop(columns=['spotify_track_uri', 'track_name', 'artist_name', 'album_name'], inplace=True)
df = df.iloc[:, [-1, 0, 1, 2, 3, 4, 5, 6]]

In [63]:
df.head(3)

,song_id,ts,platform,ms_played,reason_start,reason_end,shuffle,skipped
0,0,2013-07-08 02:44:34,web player,3185,autoplay,clickrow,False,False
1,1,2013-07-08 02:45:37,web player,61865,clickrow,clickrow,False,False
2,2,2013-07-08 02:50:24,web player,285386,clickrow,unknown,False,False


In [62]:
song_info.head(3)

,song_id,spotify_track_uri,track_name,artist_name,album_name
0,0,2J3n32GeLmMjwuAzyhcSNe,"Say It, Just Say It",The Mowgli's,Waiting For The Dawn
1,1,1oHxIPqJyvAYHy0PVrDU98,Drinking from the Bottle (feat. Tinie Tempah),Calvin Harris,18 Months
2,2,487OPlneJNni3NWC8SYqhW,Born To Die,Lana Del Rey,Born To Die - The Paradise Edition


In [59]:
df.to_csv('spotify_history.csv', index=False)
song_info.to_csv('spotify_songs.csv', index=False)